<a href="https://colab.research.google.com/github/FranzDS01/InteligenciaArtifical/blob/main/Redes%20neuronales/Final_Redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Plantilla de redes neuronales**

**Nombre: ** Soria Colque Franz Ronald

**Carrera:** Ingenieria de Sistemas

**Repositorio:**

---

**Dataset implementado:** Reconocimiento de actividad humana con teléfonos inteligentes

 [Sitio de desgarga del dataset](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones)

<p>Descripcion del dataset</p>
La base de datos de reconocimiento de actividad humana se creó a partir de las grabaciones de 30 participantes del estudio que realizaban actividades de la vida diaria (ADL) mientras llevaban un teléfono inteligente montado en la cintura con sensores de inercia integrados. El objetivo es clasificar las actividades en una de las seis actividades realizadas .
<p>actividades (CAMINAR, CAMINAR ARRIBA, CAMINAR ABAJO, SENTADO, DE PIE, ACOSTADO)</p>

**Detalles de la solucion:**
El dataset cuenta con datos de entrenamiento y datos de prueba

<table>
  <tr>
    <th>data</th>
    <th>Entrenamiento</th>
    <th>Prueba</th>
  </tr>
  <tr>
    <td>Filas</td>
    <td>7352</td>
    <td>2947</td>
  </tr>
  <tr>
    <td>Columnas</td>
    <td>563</td>
    <td>563</td>
  </tr>
</table>

<p>Para poder trabajar con el dataset, se verificaron los siguientes aspectos:</p>
<ul>
  <li><strong>Datos Nulos:</strong> No cuenta con datos nulos.</li>
  <li><strong>Datos categóricos:</strong> Sí cuenta con una columna categórica [Activity] que es la columna objetivo.</li>
  <li><strong>Columnas innecesarias:</strong> Ninguna.</li>
</ul>

<p>Antes de convertir la columna <span style="color:orange">[Activity]</span> a numérica, primero se obtuvieron los valores únicos de esa columna en la variable <code>value=['STANDING','SITTING','LAYING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']</code>.</p>

traducida al español value=['De pie','Sentado','Acostado','Caminar','Caminar Abajo','Caminar Arriba'] que sera usada al final para mostrar las predicciones


Ya verificados se hiso el tratado de datos conviertiendo la columna [Activity] a numerica implementado la funcion **tratadoDatos(dataset,modificarColumnas,eliminarColumnas):**

Luego se Asigno todas las columnas x a  "X" y la columna [Activity] a "Y"


Para entrenar el modelo, se utilizaron 33 neuronas y 2 capas ocultas con la función de activación ReLU. En la capa de salida, se utilizaron las funciones de activación sigmoide y softmax. Se utilizó una tasa de aprendizaje de 0.01 y la función de pérdida CrossEntropy para un problema de clasificación. El modelo se entrenó durante 200 épocas con un tamaño de lote definido por batch_size =15 . Con estos parámetros, el modelo se entrenó de manera efectiva en un tiempo razonable.

Para hacer la predicción, se utilizó un conjunto de datos de prueba con un preprocesamiento diferente al utilizado en el conjunto de datos de entrenamiento. Esto se hizo para evitar modificar las clasificaciones en la columna Activity al convertirlas en valores numéricos.

`(ejemplo: colummnas Activity de data de entrenamiento primer valor "LAYING" valor unico =0
columna Activity de data de prueba primer valor "STADING" valor unico 0
se estan cambiando las clases , es por eso que no se usa el mismo tratado de datos
)`

 En su lugar, se utilizó una función llamada reemplazar_valores_categoricos para convertir la columna Activity en valores numéricos utilizando las categorías(Value) obtenidas del conjunto de datos de entrenamiento.

Esta función toma como entrada el nombre de la columna Activity y las categorías obtenidas del conjunto de datos de entrenamiento. Las categorías son una lista ordenada de clases únicas, como ['STANDING','SITTING','LAYING','WALKING', 'WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']. La función compara cada valor en la columna Activity del conjunto de datos de prueba con las categorías y asigna el índice correspondiente a cada valor. Por ejemplo, si el primer valor en la columna Activity es LAYING, la función buscaría esta palabra en las categorías y encontraría que está en el índice 2. Luego, reemplazaría el valor LAYING por 2 en la columna Activity. De esta manera, se asigna el valor numérico correspondiente a cada clase.


La precisión del modelo con los datos de prueba superó el 90% [93.41%]. Para visualizar las predicciones y compararlas con los datos reales, se está utilizando la función resultados(prediccion, y_real, value). Esta función muestra en verde las predicciones que fueron correctas y en rojo las que fallaron.



In [981]:
import os
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot
from sklearn.model_selection import train_test_split # libreria usado para separar los datos en entrenamiento y prueba
%matplotlib inline
np.random.seed(0)

# Clases y funciones de redes neuronales

In [982]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [983]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [984]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa peso inicial diferente en cada ejecucion
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [985]:
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [986]:
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [987]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target.astype(int)]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target.astype(int)] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

# Funciones para el tratado de datos

In [988]:
def tratadoDatos(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')
    columna = data.columns
    #Convierte columnas categoricas a numericas
    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))])))
    #elimina columnas
    data = data.drop([columna[index]for index in eliminarColumnas], axis=1)
    #rellena datos nulos con la media
    valor_media=data.select_dtypes(include=['number']).mean()
    data = data.fillna(valor_media.round(2))
    return data

In [989]:
def datosSinteticos(data,listaData,n_filas,clases):
    #Crear nuevos_datos con n filas nuevas y misma cantidad de columnas del dataFrame
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    #Genera datos sinteticos aleotorios en funcion al numero de clases para la columnas Y
    ColmunaY = np.random.randint(0,clases, n_filas)
    #Genera el resto de los datos para las columnas X en funcion al ColumnaY
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            pos=ColmunaY[j]
            columna_i =listaData[pos].iloc[:,i]
            muestras_i=np.random.choice(columna_i,1,replace=True)

            pro_cambio = random.random()
            decision = random.randint(0, 1)
            if pro_cambio < 0.2:
               muestras_i = muestras_i + pro_cambio if decision == 0 else muestras_i - pro_cambio

            if(i == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(2)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,len(data.columns)-1]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_data=pd.DataFrame(nuevos_datos,columns=columnas)
    #Nuevo dataFrame con los datos orginales y sinteticas
    dataExtend =pd.concat([data,nuevos_datos_data],axis=0)
    #print(dataExtend.tail())
    return dataExtend

In [990]:
def listaDataframes(data,clases,columna_y):
    #Lista de dataFrames separados en funcion a su clase
    ListaData=[]
    for i in range(0,clases):
        ListaData.append(data[data[columna_y] == i])
    return ListaData

In [991]:
def categoricas(data):
    #Array con las posiciones de columnas categoricas del dataset
    columna_categorica = data.select_dtypes(include=['object'])
    print(columna_categorica.columns)
    categorica = np.array([data.columns.get_loc(col) for col in columna_categorica.columns])
    return categorica

In [992]:
def clasificaciones(data,columnay):
    #Vista de Clasificaciones su cantidad del la columnas objetivo
    value = list(data[columnay].unique())
    print("clases :",value)
    print('Cantidad:',len(value))
    data.info()
    return value

In [993]:
def reorganizarDataset(data, posicion_columna):
    #Mover la columnas Objetivo a final
    # Obtener el nombre de la columna a mover
    nombre_columna = data.columns[posicion_columna]
    # Crear una lista con las columnas en el nuevo orden
    columnas_nuevas = list(data.columns[:posicion_columna]) + list(data.columns[posicion_columna+1:]) + [nombre_columna]
    data_reorganizado = data[columnas_nuevas]
    return data_reorganizado

In [994]:
def nulos(data):
    #cantidad de nulos y nombre y posicion de las columnas que lo tengan
    nulos = data.isnull().sum().sum()
    print("cantidad de nulos :",nulos)
    for i in range(len(data.columns)):
        if data[data.columns[i]].isnull().sum()>0:
            print(data.columns[i],i)

In [995]:
def segmantacion(data,columnay):
    #separacion en datos de prueba y entrenamiento
    X = data.drop(columnay, axis=1)
    y = data[columnay]
    X_entre, X_prueba, y_entre, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Entrenamiento:',X_entre.shape,y_entre.shape,'Prueba:',X_prueba.shape,y_prueba.shape)
    return X_entre.to_numpy(), y_entre.to_numpy(),X_prueba.to_numpy(),y_prueba.to_numpy()

In [996]:
def  normalizacion(X):
    #normaliza datos
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma
    print('Media calculada:', mu)
    print('Desviación estandar calculada:', sigma)
    return X_norm, mu, sigma

# Desarrollo de la solucion

In [997]:
data=pd.read_csv('/content/train.csv')
data

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819,30,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053,30,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811,30,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339,30,WALKING_UPSTAIRS


In [998]:
categorica=categoricas(data)
print(categorica)
nulos(data)

Index(['Activity'], dtype='object')
[562]
cantidad de nulos : 0


In [999]:
value=clasificaciones(data,'Activity')

clases : ['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']
Cantidad: 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [1000]:
data=tratadoDatos('/content/train.csv',categorica,[])

In [1001]:
data=data.to_numpy()
X=data[:,:562]
Y=data[:,562]
print(X.shape)
print(Y.shape)
print(X[0])
print("---"*20)
print(Y)

(7352, 562)
(7352,)
[ 0.28858451 -0.02029417 -0.13290514 -0.9952786  -0.98311061 -0.91352645
 -0.99511208 -0.98318457 -0.92352702 -0.93472378 -0.56737807 -0.74441253
  0.85294738  0.68584458  0.81426278 -0.96552279 -0.99994465 -0.99986303
 -0.99461218 -0.99423081 -0.98761392 -0.94321999 -0.40774707 -0.67933751
 -0.60212187  0.92929351 -0.85301114  0.35990976 -0.05852638  0.25689154
 -0.22484763  0.26410572 -0.09524563  0.27885143 -0.46508457  0.49193596
 -0.19088356  0.37631389  0.43512919  0.66079033  0.96339614 -0.14083968
  0.11537494 -0.98524969 -0.98170843 -0.87762497 -0.98500137 -0.98441622
 -0.89467735  0.89205451 -0.16126549  0.12465977  0.97743631 -0.12321341
  0.05648273 -0.37542596  0.89946864 -0.97090521 -0.97551037 -0.98432539
 -0.98884915 -0.91774264 -1.         -1.          0.11380614 -0.590425
  0.5911463  -0.59177346  0.59246928 -0.74544878  0.72086167 -0.71237239
  0.71130003 -0.99511159  0.99567491 -0.99566759  0.99165268  0.57022164
  0.43902735  0.98691312  0.07799

In [1002]:
D_in = 562  # Entradas X
H = 33      # Neuronas
D_out = 6   #Clasificaciones
mlp = MLP([
    Linear(D_in, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, D_out)
])

optimizer = SGD(mlp, lr=0.01)
loss = CrossEntropy(mlp)

epochs = 200   #epocas (veces que pasa el modelo por todo el conjunto de entrenamiento )
batch_size = 15   #Lotes de entrenamiento(Numero de muestras) que se usan para actualizar los pesos del modelo durante el entrenamiento

batches = len(X) // batch_size
log_each = 40  # cada cuantas epocas se va mostrar la perdida promedio
l = []
for e in range(1,epochs+1):
    _l = []
    for b in range(batches):
        x = X[b*batch_size:(b+1)*batch_size]
        y = Y[b*batch_size:(b+1)*batch_size]
        y_pred = mlp(x)
        _l.append(loss(y_pred, y))
        loss.backward()
        optimizer.update()
    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')


Epoch 40/200, Loss: 0.3259
Epoch 80/200, Loss: 0.2214
Epoch 120/200, Loss: 0.1742
Epoch 160/200, Loss: 0.1460
Epoch 200/200, Loss: 0.1264


### Probar con datos de prueba

In [1003]:
data=pd.read_csv('/content/test.csv')

In [1004]:
def reemplazar_valores_categoricos(data, columnas, categorias):
    #solo datos de entrenamiento
    for col, cat in zip(columnas, categorias):
        data[col] = data[col].map(lambda x: cat.index(x))
    return data

In [1005]:
value

['STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS']

In [1006]:
columnas = ['Activity']
categorias=[value]
data = reemplazar_valores_categoricos(data, columnas, categorias)

In [1007]:
categoricas(data)

Index([], dtype='object')


array([], dtype=float64)

In [1008]:
data=data.to_numpy()
X=data[:,:562]
Y=data[:,562]
print(X.shape)
print(Y.shape)

(2947, 562)
(2947,)


In [1009]:
#Actividades (CAMINAR, CAMINAR_ARRIBA, CAMINAR_ABAJO, SENTADO, DE PIE, ACOSTADO)
#(WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING)
value

['STANDING',
 'SITTING',
 'LAYING',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS']

In [1010]:
value=['De pie','Sentado','Acostado','Caminar','Caminar Abajo','Caminar Arriba']

In [1011]:
from IPython.display import HTML
#Imprimir resultados de prediccion y  las reales
def resultados(prediccion,y_real,value):
  display(HTML('<b>N  &nbsp &nbsp Reales  &nbsp &nbsp &nbsp  Predicciones</b>'))
  num=1
  for p,y in zip(prediccion , y_real):
      if p == y:
          display(HTML(f'<font color="green"> {num}&nbsp &nbsp    {value[int(y)]}   &nbsp &nbsp &nbsp      {value[int(p)]}'))
      else:
          display(HTML(f'<font color="red">  {num}  &nbsp &nbsp  {value[int(y)]}   &nbsp &nbsp &nbsp      {value[int(p)]}</font>'))
      num=num+1;

In [1012]:
#Array de prueba
cantidad_pruebas=2946

X_test = X[:cantidad_pruebas, :].copy()

#Normalizar los datos de prueba si es nesesario
#X_test_norm = (X_test - X_mean) / X_std

# Pasar los datos de prueba a través del modelo para obtener las predicciones
prediccion_y = mlp(X_test)

#encuentra la clase con la mayor probabilidad en las predicciones del modelo
prediccion = np.argmax(prediccion_y, axis=1)

# Porcentaje de relacion entre prediccion y datos reales
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(np.mean(prediccion == Y[:cantidad_pruebas]) * 100))

# Vista de Predicciones del modelo para comparar
y_real=Y[:cantidad_pruebas]

#resultados(prediccion,y_real,value)#93.41%


Precisión del conjunto de entrenamiento: 93.41%
